# Illustrate the Effect of Varying SA Locations 

In [ ]:
from src.data.preprocess_utils import filter 
from sklearn.decomposition import FastICA
from statsmodels.tsa.stattools import acf
from matplotlib import pyplot as plt 
import seaborn as sns 
import pickle 
import os 

sns.set_theme()

### Read data 

In [5]:
with open("simulated_data_new.pkl", "rb") as file: 
    data_no_offset = pickle.load(file)
file.close()
with open("simulated_data.pkl", "rb") as file: 
    data_offset = pickle.load(file)
file.close()

obs_no_offset = data_no_offset['obs_signal']
obs_offset = data_offset['obs_signal']

### Find ICA component with highest ACF at 50 Hz 

### Compare Preprocessing 